<a href="https://colab.research.google.com/github/luisdiaz1997/DataScienceCourse/blob/master/Week_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import keras
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torchvision
from tqdm.autonotebook import tqdm

In [0]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train[:, None, :, :]/255.0
x_test = x_test[:, None, :, :]/255.0

In [0]:
plt.figure(1, figsize = (8, 8))
numbers = np.random.randint(1000, size = 16)
for i, n in enumerate(numbers):
    plt.subplot(4,4, i +1)
    plt.imshow(x_train[n, 0], cmap = 'gray')
    plt.title(y_train[n])
    plt.axis("off")

In [0]:
def imshow(img, one_channel = True):
    npimg = img.cpu().numpy()
    if one_channel:
      npimg = np.squeeze(npimg)
      plt.imshow(npimg, cmap = 'gray')
    else:
      plt.imshow(np.transpose(npimg, axes = (1, 2, 0)))

In [0]:
class Neural(torch.nn.Module):
    def __init__(self):
        super(Neural, self).__init__()
       pass
        
        
    def forward(self, inputs):
   
        
        pass

In [0]:
from torchsummary import summary


In [0]:
%load_ext tensorboard

In [0]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here


In [0]:
%tensorboard --logdir=runs

In [0]:
def images_to_probs(model, images):
    '''
    Generates predictions and corresponding probabilities from a trained
    network and a list of images
    '''
    output = model(images)
    output = torch.softmax(output, 1)

    probs, preds = torch.max(output, 1)

    
    
    return probs, preds

In [0]:
def plot_classes_preds(model, images, labels):
    probs, preds = images_to_probs( model, images)
    # plot the images in the batch, along with predicted and true labels
    fig = plt.figure(figsize=(10, 4))
    for idx in range(4):
        ax = fig.add_subplot(1, 4, idx+1, xticks=[], yticks=[])
        if images.shape[1] == 3:
          imshow(images[idx],one_channel= False)
        else:
          imshow(images[idx])
        ax.set_title("{0}, {1:.1f}%\n(label: {2})".format(
            preds[idx],
            probs[idx]*100,
            labels[idx]),
                    color=("green" if preds[idx]==labels[idx].item() else "red"))
    return fig

In [0]:
def fit(model, dataloader, loss, optimizer, device, epochs= 10, print_every = 100):

    model.train()
    for epoch in tqdm(range(epochs)):
      running_cost = 0.0
      for i, data in enumerate(tqdm(dataloader, leave = False)):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        J = loss(outputs, labels)
        J.backward()
        optimizer.step()
        running_cost += J.item()
        if i % print_every == (print_every-1):
          writer.add_scalar('training_cost',
                            running_cost / print_every,
                            epoch * len(dataloader) + i)
          
          writer.add_figure('predictions vs. actuals',
                            plot_classes_preds(model, inputs, labels),
                            global_step=epoch * len(dataloader) + i)
          running_cost = 0.0

In [0]:
def accuracy(model, dataloader, device):
  model.eval()
  correct = 0
  total = 0
  with torch.no_grad():
      for data in dataloader:
          images, labels = data[0].to(device), data[1].to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  print('Accuracy of the network on the 10000 test images: %d %%' % (
      100 * correct / total))

In [0]:
! pip install -q kaggle
! mkdir ~/.kaggle

In [0]:
! cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d chetankv/dogs-cats-images
!unzip dogs-cats-images.zip


In [0]:
writer = SummaryWriter('runs/cat_dog')